##### Importing Modules


In [5]:
from abc import ABC, abstractmethod
from typing import Literal, Tuple, List, Callable, Optional
import math
import time
from typing import TypeVar

##### Invalid Input Custom Exception


In [6]:
class InvalidInput(Exception):
    def __init__(self, message="Invalid Input Enter, Please Try Again"):
        self.message = message
        super().__init__(self.message)

##### Invalid Position Custom Exception


In [7]:
class InvalidPosition(Exception):
    def __init__(self, message="Invalid Position Enter, Please Choose Your Piece Only"):
        self.message = message
        super().__init__(self.message)

##### Invalid Move Custom Exception


In [8]:
class InvalidMove(Exception):
    def __init__(self, message="Invalid Move Enter, Please Enter Valid Move"):
        self.message = message
        super().__init__(self.message)

##### Type Hints


In [9]:
ALPHABET_COL_TYPE = Literal["a", "b", "c", "d", "e", "f", "g", "h"]
NUM_COL_TYPE = Literal[1, 2, 3, 4, 5, 6, 7, 8]
NUM_ROW_TYPE = Literal[1, 2, 3, 4, 5, 6, 7, 8]
PLAYERS_TYPE = Literal["white", "black"]

##### Constants


In [10]:
ALPHABET_COL = ["a", "b", "c", "d", "e", "f", "g", "h"]
NUM_COL = [1, 2, 3, 4, 5, 6, 7, 8]
NUM_ROW = [1, 2, 3, 4, 5, 6, 7, 8]
PLAYERS = ["white", "black"]

##### Alphabet Column to Numerical Column Mapper


In [11]:
CHESS_ALPHABET_MAPPING = {
    "a": 1,
    "b": 2,
    "c": 3,
    "d": 4,
    "e": 5,
    "f": 6,
    "g": 7,
    "h": 8,
}


def get_alphabet(num: int):
    for k, v in CHESS_ALPHABET_MAPPING.items():
        if v == num:
            return k
    return False

##### Pieces Base Abstract Class


In [12]:
class Piece(ABC):

    def __init__(self, player: PLAYERS_TYPE):
        self.player: PLAYERS_TYPE = player

    # @abstractmethod
    # def possible_moves(self,current_position:Tuple[NUM_COL_TYPE,NUM_ROW_TYPE]):
    #     pass

    @abstractmethod
    def __str__(self) -> str:
        pass

##### Piece Type Hints


In [13]:
PIECE_TYPE = TypeVar("Piece", bound=Piece)

##### Pawn Child Class


In [14]:
class Pawn(Piece):

    def __init__(self, player: PLAYERS_TYPE):
        self.is_first_move: bool = True
        super().__init__(player=player)

    def get_possible_moves_function(self):

        def get_possible_moves(
            position: Tuple[NUM_ROW_TYPE, NUM_COL_TYPE],
            is_valid_position,
            is_piece,
            is_player_piece,
        ):
            possible_moves_list: List[Tuple[NUM_ROW_TYPE, NUM_COL_TYPE]] = []
            if self.player == "white":
                # For first move
                if self.is_first_move:
                    move_position = (position[0] + 2, position[1])
                    if is_valid_position(move_position) and not is_piece(move_position):
                        possible_moves_list.append(move_position)
                # For normal move
                move_position = (position[0] + 1, position[1])
                if is_valid_position(move_position) and not is_piece(move_position):
                    possible_moves_list.append(move_position)
                # For left forward kill
                move_position = (position[0] + 1, position[1] - 1)
                if is_valid_position(move_position) and is_player_piece(
                    player="black", position=move_position
                ):
                    possible_moves_list.append(move_position)
                # For right forward kill
                move_position = (position[0] + 1, position[1] + 1)
                if is_valid_position(move_position) and is_player_piece(
                    player="black", position=move_position
                ):
                    possible_moves_list.append(move_position)
            elif self.player == "black":
                # For first move
                if self.is_first_move:
                    move_position = (position[0] - 2, position[1])
                    if is_valid_position(move_position) and not is_piece(move_position):
                        possible_moves_list.append(move_position)
                # For normal move
                move_position = (position[0] - 1, position[1])
                if is_valid_position(move_position) and not is_piece(move_position):
                    possible_moves_list.append(move_position)
                # For left forward kill
                move_position = (position[0] - 1, position[1] - 1)
                if is_valid_position(move_position) and is_player_piece(
                    player="white", position=move_position
                ):
                    possible_moves_list.append(move_position)
                # For right forward kill
                move_position = (position[0] - 1, position[1] + 1)
                if is_valid_position(move_position) and is_player_piece(
                    player="white", position=move_position
                ):
                    possible_moves_list.append(move_position)

            return possible_moves_list

        return get_possible_moves

    def get_move_piece_function(self):

        def move_piece(
            board,
            current_position: Tuple[NUM_ROW_TYPE, NUM_COL_TYPE],
            move_position: Tuple[NUM_ROW_TYPE, NUM_COL_TYPE],
        ):
            if self.is_first_move:
                self.is_first_move = False
            board[move_position[0]][move_position[1]] = board[current_position[0]][
                current_position[1]
            ]
            board[current_position[0]][current_position[1]] = None

        return move_piece

    def __str__(self) -> str:
        if self.player == "white":
            return "♟"
        elif self.player == "black":
            return "♙"

##### Rook Child Class


In [15]:
class Rook(Piece):

    def __init__(self, player: PLAYERS_TYPE):
        super().__init__(player=player)

    def get_possible_moves_function(self):

        def get_possible_moves(
            position: Tuple[NUM_ROW_TYPE, NUM_COL_TYPE],
            is_valid_position,
            is_piece,
            is_player_piece,
        ):
            current_row = position[0]
            current_col = position[1]
            enemy_player = "white" if self.player == "black" else "black"
            possible_moves_list: List[Tuple[NUM_ROW_TYPE, NUM_COL_TYPE]] = []

            # For vertical moves

            # For forward moves
            for row in range(current_row + 1, 9):
                move_position = (row, current_col)
                if is_piece(move_position):
                    if is_player_piece(player=enemy_player, position=move_position):
                        possible_moves_list.append(move_position)
                    break
                possible_moves_list.append(move_position)

            # For backward moves
            for row in range(current_row - 1, 0, -1):
                move_position = (row, current_col)

                if is_piece(move_position):
                    if is_player_piece(player=enemy_player, position=move_position):
                        possible_moves_list.append(move_position)
                    break
                possible_moves_list.append(move_position)

            # For horizontal moves

            # For right moves
            for col in range(current_col + 1, 9):
                move_position = (current_row, col)
                if is_piece(move_position):
                    if is_player_piece(player=enemy_player, position=move_position):
                        possible_moves_list.append(move_position)
                    break
                possible_moves_list.append(move_position)

            # For left moves
            for col in range(current_col - 1, 0, -1):
                move_position = (current_row, col)
                if is_piece(move_position):
                    if is_player_piece(player=enemy_player, position=move_position):
                        possible_moves_list.append(move_position)
                    break
                possible_moves_list.append(move_position)

            return possible_moves_list

        return get_possible_moves

    def __str__(self) -> str:
        if self.player == "white":
            return "♜"
        elif self.player == "black":
            return "♖"

##### Knight Child Class


In [16]:
class Knight(Piece):

    def __init__(self, player: PLAYERS_TYPE):
        super().__init__(player=player)

    def get_possible_moves_function(self):

        def get_possible_moves(
            position: Tuple[NUM_ROW_TYPE, NUM_COL_TYPE],
            is_valid_position,
            is_piece,
            is_player_piece,
        ):
            current_row = position[0]
            current_col = position[1]
            possible_moves_list: List[Tuple[NUM_ROW_TYPE, NUM_COL_TYPE]] = []

            # L shape up
            # For left 
            move_position = (current_row+2,current_col-1)
            if is_valid_position(move_position) and not is_player_piece(player=self.player, position=move_position):
                possible_moves_list.append(move_position)

            # For right
            move_position = (current_row+2,current_col+1)
            if is_valid_position(move_position) and not is_player_piece(player=self.player, position=move_position):
                possible_moves_list.append(move_position)

            # L shape down
            # For left 
            move_position = (current_row-2,current_col-1)
            if is_valid_position(move_position) and not is_player_piece(player=self.player, position=move_position):
                possible_moves_list.append(move_position)

            # For right
            move_position = (current_row-2,current_col+1)
            if is_valid_position(move_position) and not is_player_piece(player=self.player, position=move_position):
                possible_moves_list.append(move_position)

            # L flipped shape up
            # For left 
            move_position = (current_row+1,current_col-2)
            if is_valid_position(move_position) and not is_player_piece(player=self.player, position=move_position):
                possible_moves_list.append(move_position)

            # For right
            move_position = (current_row+1,current_col+2)
            if is_valid_position(move_position) and not is_player_piece(player=self.player, position=move_position):
                possible_moves_list.append(move_position)

            # L flipped shape down
            # For left 
            move_position = (current_row-1,current_col-2)
            if is_valid_position(move_position) and not is_player_piece(player=self.player, position=move_position):
                possible_moves_list.append(move_position)

            # For right
            move_position = (current_row-1,current_col+2)
            if is_valid_position(move_position) and not is_player_piece(player=self.player, position=move_position):
                possible_moves_list.append(move_position)

            return possible_moves_list

        return get_possible_moves


    def __str__(self) -> str:
        if self.player == "white":
            return "♞"
        elif self.player == "black":
            return "♘"

##### Bishop Child Class


In [17]:
class Bishop(Piece):

    def __init__(self, player: PLAYERS_TYPE):
        super().__init__(player=player)

    def get_possible_moves_function(self):

        def get_possible_moves(
            position: Tuple[NUM_ROW_TYPE, NUM_COL_TYPE],
            is_valid_position,
            is_piece,
            is_player_piece,
        ):
            current_row = position[0]
            current_col = position[1]
            enemy_player = "white" if self.player == "black" else "black"
            possible_moves_list: List[Tuple[NUM_ROW_TYPE, NUM_COL_TYPE]] = []

            # For left moves

            # For left upward moves
            row = current_row + 1
            col = current_col - 1
            move_position = (row, col)
            while is_valid_position(move_position):
                if is_piece(move_position):
                    if is_player_piece(player=enemy_player, position=move_position):
                        possible_moves_list.append(move_position)
                    break
                possible_moves_list.append(move_position)
                row += 1
                col -= 1
                move_position = (row, col)

            # For left downward moves
            row = current_row - 1
            col = current_col - 1
            move_position = (row, col)
            while is_valid_position(move_position):
                if is_piece(move_position):
                    if is_player_piece(player=enemy_player, position=move_position):
                        possible_moves_list.append(move_position)
                    break
                possible_moves_list.append(move_position)
                row -= 1
                col -= 1
                move_position = (row, col)

            # For right moves

            # For right upward moves
            row = current_row + 1
            col = current_col + 1
            move_position = (row, col)
            while is_valid_position(move_position):
                if is_piece(move_position):
                    if is_player_piece(player=enemy_player, position=move_position):
                        possible_moves_list.append(move_position)
                    break
                possible_moves_list.append(move_position)
                row += 1
                col += 1
                move_position = (row, col)

            # For right downward moves
            row = current_row - 1
            col = current_col + 1
            move_position = (row, col)
            while is_valid_position(move_position):
                if is_piece(move_position):
                    if is_player_piece(player=enemy_player, position=move_position):
                        possible_moves_list.append(move_position)
                    break
                possible_moves_list.append(move_position)
                row -= 1
                col += 1
                move_position = (row, col)

            return possible_moves_list

        return get_possible_moves

    def __str__(self) -> str:
        if self.player == "white":
            return "♝"
        elif self.player == "black":
            return "♗"

##### Queen Child Class


In [18]:
class Queen(Piece):

    def __init__(self, player: PLAYERS_TYPE):
        super().__init__(player=player)

    def get_possible_moves_function(self):

        def get_possible_moves(
            position: Tuple[NUM_ROW_TYPE, NUM_COL_TYPE],
            is_valid_position,
            is_piece,
            is_player_piece,
        ):
            current_row = position[0]
            current_col = position[1]
            enemy_player = "white" if self.player == "black" else "black"
            possible_moves_list: List[Tuple[NUM_ROW_TYPE, NUM_COL_TYPE]] = []

            # For rook moves
            # For vertical moves

            # For forward moves
            for row in range(current_row + 1, 9):
                move_position = (row, current_col)
                if is_piece(move_position):
                    if is_player_piece(player=enemy_player, position=move_position):
                        possible_moves_list.append(move_position)
                    break
                possible_moves_list.append(move_position)

            # For backward moves
            for row in range(current_row - 1, 0, -1):
                move_position = (row, current_col)

                if is_piece(move_position):
                    if is_player_piece(player=enemy_player, position=move_position):
                        possible_moves_list.append(move_position)
                    break
                possible_moves_list.append(move_position)

            # For horizontal moves

            # For right moves
            for col in range(current_col + 1, 9):
                move_position = (current_row, col)
                if is_piece(move_position):
                    if is_player_piece(player=enemy_player, position=move_position):
                        possible_moves_list.append(move_position)
                    break
                possible_moves_list.append(move_position)

            # For left moves
            for col in range(current_col - 1, 0, -1):
                move_position = (current_row, col)
                if is_piece(move_position):
                    if is_player_piece(player=enemy_player, position=move_position):
                        possible_moves_list.append(move_position)
                    break
                possible_moves_list.append(move_position)

            # Bishop Moves
            # For left moves

            # For left upward moves
            row = current_row + 1
            col = current_col - 1
            move_position = (row, col)
            while is_valid_position(move_position):
                if is_piece(move_position):
                    if is_player_piece(player=enemy_player, position=move_position):
                        possible_moves_list.append(move_position)
                    break
                possible_moves_list.append(move_position)
                row += 1
                col -= 1
                move_position = (row, col)

            # For left downward moves
            row = current_row - 1
            col = current_col - 1
            move_position = (row, col)
            while is_valid_position(move_position):
                if is_piece(move_position):
                    if is_player_piece(player=enemy_player, position=move_position):
                        possible_moves_list.append(move_position)
                    break
                possible_moves_list.append(move_position)
                row -= 1
                col -= 1
                move_position = (row, col)

            # For right moves

            # For right upward moves
            row = current_row + 1
            col = current_col + 1
            move_position = (row, col)
            while is_valid_position(move_position):
                if is_piece(move_position):
                    if is_player_piece(player=enemy_player, position=move_position):
                        possible_moves_list.append(move_position)
                    break
                possible_moves_list.append(move_position)
                row += 1
                col += 1
                move_position = (row, col)

            # For right downward moves
            row = current_row - 1
            col = current_col + 1
            move_position = (row, col)
            while is_valid_position(move_position):
                if is_piece(move_position):
                    if is_player_piece(player=enemy_player, position=move_position):
                        possible_moves_list.append(move_position)
                    break
                possible_moves_list.append(move_position)
                row -= 1
                col += 1
                move_position = (row, col)

            return possible_moves_list

        return get_possible_moves

    def __str__(self) -> str:
        if self.player == "white":
            return "♛"
        elif self.player == "black":
            return "♕"

##### King Child Class


In [19]:
class King(Piece):

    def __init__(self, player: PLAYERS_TYPE):
        super().__init__(player=player)

    def get_possible_moves_function(self):

        def get_possible_moves(
            position: Tuple[NUM_ROW_TYPE, NUM_COL_TYPE],
            is_valid_position,
            is_piece,
            is_player_piece,
        ):
            current_row = position[0]
            current_col = position[1]
            possible_moves_list: List[Tuple[NUM_ROW_TYPE, NUM_COL_TYPE]] = []

            # For up
            move_position = (current_row+1,current_col)
            if is_valid_position(move_position) and not is_player_piece(player=self.player, position=move_position):
                possible_moves_list.append(move_position)

            # For down
            move_position = (current_row-1,current_col)
            if is_valid_position(move_position) and not is_player_piece(player=self.player, position=move_position):
                possible_moves_list.append(move_position)

            # For left
            move_position = (current_row,current_col-1)
            if is_valid_position(move_position) and not is_player_piece(player=self.player, position=move_position):
                possible_moves_list.append(move_position)

            # For right
            move_position = (current_row,current_col+1)
            if is_valid_position(move_position) and not is_player_piece(player=self.player, position=move_position):
                possible_moves_list.append(move_position)

            # For left upward
            move_position = (current_row+1,current_col-1)
            if is_valid_position(move_position) and not is_player_piece(player=self.player, position=move_position):
                possible_moves_list.append(move_position)

            # For left downward
            move_position = (current_row-1,current_col-1)
            if is_valid_position(move_position) and not is_player_piece(player=self.player, position=move_position):
                possible_moves_list.append(move_position)

            # For right upward
            move_position = (current_row+1,current_col+1)
            if is_valid_position(move_position) and not is_player_piece(player=self.player, position=move_position):
                possible_moves_list.append(move_position)

            # For right downward
            move_position = (current_row-1,current_col+1)
            if is_valid_position(move_position) and not is_player_piece(player=self.player, position=move_position):
                possible_moves_list.append(move_position)

            return possible_moves_list

        return get_possible_moves

    def __str__(self) -> str:
        if self.player == "white":
            return "♚"
        elif self.player == "black":
            return "♔"

##### Main Chess Class


In [20]:
from IPython.display import clear_output


class Chess:

    def __init__(self):
        self.board = self.init_board()
        self.place_pieces()

    @staticmethod
    def print_heading(heading: str) -> None:
        length: int = len(heading)
        gap: int = 60 - length
        left_gap: int = math.floor(gap / 2)
        right_gap: int = math.ceil(gap / 2)
        print(f"{" "*left_gap}{heading}{" "*right_gap}")

    @staticmethod
    def print_piece(piece: Optional[PIECE_TYPE]) -> None:
        if piece is None:
            print("      ", end="")
        else:
            if type(piece) is str:
                length: int = len(piece)
            else:
                length: int = 1
            gap: int = 6 - length
            left_gap: int = math.floor(gap / 2)
            right_gap: int = math.ceil(gap / 2)
            print(f"{' '*left_gap}{piece}{' '*right_gap}", end="")
        print(end="|")

    @staticmethod
    def get_position_str(position: Tuple[NUM_ROW_TYPE, NUM_COL_TYPE]):
        return f"{get_alphabet(position[1])+str(position[0])}"

    @staticmethod
    def is_valid_input(entered_input: str) -> bool:
        if type(entered_input) is not str:
            return False
        if len(entered_input) != 2:
            return False
        if type(entered_input[0]) is str and entered_input[0] not in ALPHABET_COL:
            return False
        if entered_input[1].isdigit() and int(entered_input[1]) not in NUM_ROW:
            return False
        return True

    @staticmethod
    def is_valid_position(position: Tuple[NUM_ROW_TYPE, NUM_COL_TYPE]):
        if position[0] in NUM_ROW and position[1] in NUM_COL:
            return True
        else:
            return False

    def print_piece_moves(
        self,
        position: Tuple[NUM_ROW_TYPE, NUM_COL_TYPE],
        possible_moves_list: List[Tuple[NUM_ROW_TYPE, NUM_COL_TYPE]],
    ):
        piece = self.board[position[0]][position[1]]
        possible_moves_str = ""
        for pos in possible_moves_list:
            possible_moves_str += " | " + Chess.get_position_str(position=pos)
        print()
        if possible_moves_str == "":
            Chess.print_heading(
                f"{piece} ({Chess.get_position_str(position)}) Can Not Move. Please Change Piece"
            )
        else:
            Chess.print_heading(
                f"{piece} ({Chess.get_position_str(position)}) Can Move To {possible_moves_str} |"
            )

    def move_piece(
        self,
        current_position: Tuple[NUM_ROW_TYPE, NUM_COL_TYPE],
        move_position: Tuple[NUM_ROW_TYPE, NUM_COL_TYPE],
    ):
        piece = self.board[current_position[0]][current_position[1]]
        if isinstance(piece, Pawn):
            pawn_move_piece = piece.get_move_piece_function()
            pawn_move_piece(
                board=self.board,
                current_position=current_position,
                move_position=move_position,
            )
        else:
            self.board[move_position[0]][move_position[1]] = self.board[
                current_position[0]
            ][current_position[1]]
            self.board[current_position[0]][current_position[1]] = None

    def is_piece(self, position: Tuple[NUM_ROW_TYPE, NUM_COL_TYPE]) -> bool:

        position_value: Optional[PIECE_TYPE] = self.board[position[0]][position[1]]

        if position_value is None:
            return False
        elif isinstance(position_value, Piece):
            return True
        else:
            return False

    def is_player_piece(
        self, player: PLAYERS_TYPE, position: Tuple[NUM_ROW_TYPE, NUM_COL_TYPE]
    ) -> bool:

        position_value: Optional[PIECE_TYPE] = self.board[position[0]][position[1]]

        if position_value is None:
            return False
        elif position_value.player == player:
            return True
        else:
            return False

    @staticmethod
    def get_tuple_position(position_str: str) -> Tuple[NUM_ROW_TYPE, NUM_COL_TYPE]:
        return (int(position_str[1]), CHESS_ALPHABET_MAPPING[position_str[0]])

    def get_possible_moves(
        self, position: Tuple[NUM_ROW_TYPE, NUM_COL_TYPE]
    ) -> List[Tuple[NUM_ROW_TYPE, NUM_COL_TYPE]]:
        piece: PIECE_TYPE = self.board[position[0]][position[1]]
        get_possible_moves = piece.get_possible_moves_function()
        possible_moves_list: List[Tuple[NUM_ROW_TYPE, NUM_COL_TYPE]] = (
            get_possible_moves(
                position=position,
                is_valid_position=Chess.is_valid_position,
                is_piece=self.is_piece,
                is_player_piece=self.is_player_piece,
            )
        )
        return possible_moves_list

    def place_pieces(self):

        for player in PLAYERS:
            if player == "white":
                first_row = 1
                second_row = 2
            elif player == "black":
                first_row = 8
                second_row = 7

            for col in NUM_COL:

                # Placing Pawns
                self.board[second_row][col] = Pawn(player=player)

                # Placing Rook
                if col in (1, 8):
                    self.board[first_row][col] = Rook(player=player)

                # Placing Knight
                if col in (2, 7):
                    self.board[first_row][col] = Knight(player=player)

                # Placing Bishop
                if col in (3, 6):
                    self.board[first_row][col] = Bishop(player=player)

                # Placing Queen
                if col == 4:
                    self.board[first_row][col] = Queen(player=player)

                # Placing King
                if col == 5:
                    self.board[first_row][col] = King(player=player)

    def init_board(self):
        return {x: {y: None for y in range(1, 9)} for x in range(8, 0, -1)}

    def display_board(self):
        print("------------------------------------------------------------")
        for row_no, row in self.board.items():
            print(row_no, end="|| ")
            for _, piece in row.items():
                Chess.print_piece(piece)
            print()
            if row_no == 1:
                print("============================================================")
            else:
                print("------------------------------------------------------------")

        print(" ", end="|| ")
        for alpha in ALPHABET_COL:
            Chess.print_piece(alpha)
        print(f"\n------------------------------------------------------------")

    def start_game(self):

        is_black_player = False
        is_quit = False

        while True:

            Chess.print_heading("| CHESS GAME |")
            print()
            Chess.print_heading(" Press Q to Quit")
            print()
            game.display_board()
            print(f"\n")
            Chess.print_heading(f" { PLAYERS[is_black_player].upper()} PLAYER TURN")

            time.sleep(0.1)

            while True:
                try:
                    entered_position = input("Enter Position of Piece to Move : ")
                    # Quit on pressing Q | q
                    if entered_position.upper() == "Q":
                        is_quit = True
                        break

                    # Checking input
                    if not Chess.is_valid_input(entered_input=entered_position):
                        raise InvalidInput()

                    # getting position tuple
                    position: Tuple[NUM_ROW_TYPE, NUM_COL_TYPE] = (
                        Chess.get_tuple_position(position_str=entered_position)
                    )

                    # Checking if player selects his own piece
                    if not self.is_player_piece(
                        player=PLAYERS[is_black_player], position=position
                    ):
                        raise InvalidPosition(
                            f"Please Choose Position of {PLAYERS[is_black_player][0].upper()+PLAYERS[is_black_player][1:]} Piece"
                        )

                    # Getting possible moves of that piece
                    possible_moves_list: List[Tuple[NUM_ROW_TYPE, NUM_COL_TYPE]] = (
                        self.get_possible_moves(position=position)
                    )

                    # Printing possible moves
                    print()
                    Chess.print_heading(f"Press c to Change Piece")
                    self.print_piece_moves(
                        position=position, possible_moves_list=possible_moves_list
                    )

                    is_change_piece = False

                    while True:

                        entered_move_position = input("Enter Position to Move : ")

                        if entered_move_position.upper() == "C":
                            is_change_piece = True
                            break

                        if entered_move_position.upper() == "Q":
                            is_quit = True
                            break

                        try:

                            # Checking input
                            if not Chess.is_valid_input(
                                entered_input=entered_move_position
                            ):
                                raise InvalidInput(
                                    "Invalid Move, Please Enter Valid Move"
                                )

                            # getting move position tuple
                            move_position: Tuple[NUM_ROW_TYPE, NUM_COL_TYPE] = (
                                Chess.get_tuple_position(
                                    position_str=entered_move_position
                                )
                            )

                            if move_position not in possible_moves_list:
                                raise InvalidMove()

                            # Moving piece
                            self.move_piece(
                                current_position=position, move_position=move_position
                            )
                            break

                        except InvalidInput as e:
                            Chess.print_heading(f"{e}")

                        except InvalidMove as e:
                            Chess.print_heading(f"{e}")

                    if is_change_piece:
                        continue

                    if is_quit:
                        break

                    # input()
                    break

                except InvalidInput as e:
                    print()
                    Chess.print_heading(f"{e}")
                    continue

                except InvalidPosition as e:
                    print()
                    Chess.print_heading(f"{e}")
                    continue

            # Quiting game
            if is_quit:
                print()
                Chess.print_heading("GAME IS FINSIHED")
                break

            Chess.print_heading(str(position))

            is_black_player = True if is_black_player == False else False

            clear_output(wait=True)


game = Chess()
game.start_game()

                       | CHESS GAME |                       

                       Press Q to Quit                      

------------------------------------------------------------
8||   ♖   |  ♘   |  ♗   |  ♕   |  ♔   |  ♗   |  ♘   |  ♖   |
------------------------------------------------------------
7||   ♙   |  ♙   |  ♙   |  ♙   |  ♙   |  ♙   |  ♙   |  ♙   |
------------------------------------------------------------
6||       |      |      |      |      |      |      |      |
------------------------------------------------------------
5||       |      |      |      |      |      |      |      |
------------------------------------------------------------
4||       |      |      |      |      |      |      |      |
------------------------------------------------------------
3||       |      |      |      |      |      |      |      |
------------------------------------------------------------
2||   ♟   |  ♟   |  ♟   |  ♟   |  ♟   |  ♟   |  ♟   |  ♟   |
----------------------

In [73]:
lis = [0, 1, 2, 3]
print(lis[True])

1


In [ ]:
def add(a: int, b: int) -> int:
    return a + b


print(add("fff", "df"))

fffdf


In [ ]:
ALPHABET_COL = ["a", "b", "c", "d", "e", "f", "g", "h"]
NUM_COL = [1, 2, 3, 4, 5, 6, 7, 8]


while True:
    try:
        entered_input = input("Please Enter Input")
        if entered_input == "b":
            break
        print(entered_input)
        valid_input(entered_input)
    except InvalidInput as e:
        print(e)
    else:
        print("Ok")

a1


NameError: name 'valid_input' is not defined

##### Printing Chess Board


In [8]:
# def valid_position(position:Tuple[NUM_COL_TYPE,NUM_ROW_TYPE]):
#     if

In [ ]:
print("ad" + 2)

TypeError: can only concatenate str (not "int") to str

In [71]:
abs(-1)

1